In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [3]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## Rule-Based Bots ##

In [28]:
# config = GameConfig(splits=[StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit], sandbox=True, num_players=2, must_include=[])
config = GameConfig(splits=[StartingSplit.Starting34Split, StartingSplit.Starting34Split], sandbox=False, num_players=2, must_include=default_kingdom)
players = load_players(['DW', 'DW'], None, map_location='cpu', train=False)
agent = players[0]
# env = FullEnvironment(config, players, logger=logger)
env = DefaultEnvironment(config, players)

In [24]:
n = 10000

In [29]:
sim_stats = simulate(env, n, [], turn_log=True, action_log=False, card_log=True)

100%|██████████| 10000/10000 [06:07<00:00, 27.22it/s]
===SUMMARY===
{0: 5901, 1: 4495, 'ProvinceWins': 9996, 'Ties': 396}


In [30]:
data_path = os.path.join(project_root, 'data', 'dw34-dw34-10k-stats')

In [31]:
save(data_path, sim_stats)

## Tree Simulations ##

In [6]:
from mcts import GameTree

import pandas as pd

In [35]:
# tree_name = 'full-paper-tree-uniform'
# tree_name = 'full-paper-tree-160k-trimmed-lowc'
# tree_name = 'full-paper-tree-160k-trimmed-cdecay'
tree_name = 'full-paper-dw-tree-160k-trimmed-cdecay'

tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [7]:
tree_name = 'default-sandbox-uct-paper'
# tree_name = 'r-r-mcts-bm-hot-start-sandbox-100k-srat-uniform'
rollout_name = 'default-mlog-sandbox-paper'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [8]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [67]:
tree.skip_level = False

In [19]:
splits = [StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit]
config = GameConfig(splits=splits, sandbox=True, num_players=2, must_include=[])
# config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = load_players(['BM', 'UCT'], [rollout_path], tree=tree, rollout_type='mlog', train=False, use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=True, rollout_type='mlog', use_tree=True)[0]
# players = [agent, agent]

env = DefaultEnvironment(config, players)

In [16]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [20]:
sim_data = simulate(env, 10000, [tree], turn_log=True, action_log=False, card_log=True)
# sim_data = simulate(env, 1, [tree], turn_log=True, action_log=False, card_log=False)

100%|██████████| 10000/10000 [03:28<00:00, 48.07it/s]
===SUMMARY===
{0: 4825, 1: 5443, 'ProvinceWins': 10000, 'Ties': 268}


In [21]:
data_dir = os.path.join(project_root, 'data')
data_name = 'bm-uct-10k-data'
data_path = os.path.join(data_dir, data_name)

In [22]:
save(data_path, sim_data)

## Tree vs. Tree Simulations ##

In [67]:
uct_name = 'full-paper-tree-uniform'
# uct_name = 'full-paper-tree-uniform-160k'
suct_name = 'full-paper-tree-160k-trimmed-cdecay'
uctdw_name = 'full-paper-dw-tree-160k-trimmed-cdecay'
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [40]:
uct = GameTree.load(os.path.join(model_dir, uct_name), train=False, _skip_level=False, selection='max')

In [10]:
uctslim = GameTree.load(os.path.join(model_dir, suct_name), train=False, _skip_level=False, selection='max')

In [68]:
uctdw = GameTree.load(os.path.join(model_dir, uctdw_name), train=False, _skip_level=False, selection='max')

In [69]:
uct_agent = load_players(['UCT'], [rollout_path], tree=uct, train=False, rollout_type='mlog', use_tree=True)[0]
uctslim_agent = load_players(['UCT'], [rollout_path], tree=uctslim, train=True, rollout_type='mlog', use_tree=True)[0]
uctdw_agent = load_players(['UCT'], [rollout_path], tree=uctdw, train=True, rollout_type='mlog', use_tree=True)[0]

In [ ]:
dw_agent = load_players(['DW'], [])[0]

In [81]:
splits = [StartingSplit.Starting34Split, StartingSplit.Starting34Split]
config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = [uctdw_agent, uctdw_agent]

env = DefaultEnvironment(config, players)

In [109]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [82]:
sim_data = simulate(env, 1000, [uctdw], turn_log=False, action_log=False, card_log=False)

100%|██████████| 1000/1000 [00:25<00:00, 39.48it/s]===SUMMARY===
{0: 978, 1: 23, 'ProvinceWins': 1000, 'Ties': 1}

